In [50]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
spark.version

'3.2.1'

In [6]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv -P "/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework"

--2022-03-04 22:04:58--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.76.236
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.76.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  28.9MB/s    in 23s     

2022-03-04 22:05:22 (30.0 MB/s) - ‘/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [8]:
!wc -l fhvhv_tripdata_2021-02.csv

 11613943 fhvhv_tripdata_2021-02.csv


In [51]:
from pyspark.sql import types

In [52]:
schema = types.StructType([
    types.StructField('hvfhs_license_num',types.StringType(),True),
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.TimestampType(),True),
    types.StructField('dropoff_datetime',types.TimestampType(),True),
    types.StructField('PULocationID',types.IntegerType(),True),
    types.StructField('DOLocationID',types.IntegerType(),True),
    types.StructField('SR_Flag',types.StringType(),True)
])

In [53]:
df = spark.read \
    .option("header","true") \
    .schema(schema) \
    .csv('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework/fhvhv_tripdata_2021-02.csv')

In [54]:
df = df.repartition(24) #partitioning file into smaller chuncks so that executors in spark cluster can process file efficiently

In [22]:
df.write.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework/fhvhvparquet/')

In [55]:
df_parquet = spark.read.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework/fhvhvparquet/')

In [56]:
df_parquet.registerTempTable('fhvhv_feb')

In [57]:
#Q3
df_Q3 = spark.sql("""
SELECT 
    
    COUNT(1) AS number_records
    
FROM
    fhvhv_feb

WHERE pickup_datetime between '2021-02-15 00:00:00' and '2021-02-15 23:59:59'

""")

In [58]:
df_Q3.show()

+--------------+
|number_records|
+--------------+
|        367170|
+--------------+



In [59]:
#Q4
spark.sql("""
SELECT 
    
    pickup_datetime,
    dropoff_datetime,
    (dropoff_datetime - pickup_datetime) as trip_time
    
FROM
    fhvhv_feb

GROUP BY 1,2,3
ORDER BY 3 DESC

""").show()

+-------------------+-------------------+--------------------+
|    pickup_datetime|   dropoff_datetime|           trip_time|
+-------------------+-------------------+--------------------+
|2021-02-11 13:40:44|2021-02-12 10:39:44|INTERVAL '0 20:59...|
|2021-02-17 15:54:53|2021-02-18 07:48:34|INTERVAL '0 15:53...|
|2021-02-20 12:08:15|2021-02-21 00:22:14|INTERVAL '0 12:13...|
|2021-02-03 20:24:25|2021-02-04 07:41:58|INTERVAL '0 11:17...|
|2021-02-19 23:17:44|2021-02-20 09:44:01|INTERVAL '0 10:26...|
|2021-02-25 17:13:35|2021-02-26 02:57:05|INTERVAL '0 09:43...|
|2021-02-20 01:36:13|2021-02-20 11:16:19|INTERVAL '0 09:40...|
|2021-02-18 15:24:19|2021-02-19 01:01:11|INTERVAL '0 09:36...|
|2021-02-18 01:31:20|2021-02-18 11:07:15|INTERVAL '0 09:35...|
|2021-02-10 20:51:39|2021-02-11 06:21:08|INTERVAL '0 09:29...|
|2021-02-10 01:56:17|2021-02-10 10:57:33|INTERVAL '0 09:01...|
|2021-02-25 09:18:18|2021-02-25 18:18:57|INTERVAL '0 09:00...|
|2021-02-21 19:59:13|2021-02-22 04:56:16|INTERVAL '0 08

In [47]:
#Q5
spark.sql("""
SELECT 
    
    dispatching_base_num,
    COUNT(1) as num_of_trips
    
FROM
    fhvhv_feb

GROUP BY 1
ORDER BY 2 DESC

""").show()

+--------------------+------------+
|dispatching_base_num|num_of_trips|
+--------------------+------------+
|              B02510|     3233664|
|              B02764|      965568|
|              B02872|      882689|
|              B02875|      685390|
|              B02765|      559768|
|              B02869|      429720|
|              B02887|      322331|
|              B02871|      312364|
|              B02864|      311603|
|              B02866|      311089|
|              B02878|      305185|
|              B02682|      303255|
|              B02617|      274510|
|              B02883|      251617|
|              B02884|      244963|
|              B02882|      232173|
|              B02876|      215693|
|              B02879|      210137|
|              B02867|      200530|
|              B02877|      198938|
+--------------------+------------+
only showing top 20 rows



In [48]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv -P "/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework"

--2022-03-04 23:07:33--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.136.48
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.136.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework/taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0.001s  

2022-03-04 23:07:33 (17.5 MB/s) - ‘/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework/taxi+_zone_lookup.csv’ saved [12322/12322]



In [63]:
zones_schema=types.StructType([
    types.StructField("LocationID",types.IntegerType(),True),
    types.StructField("Borough",types.StringType(),True),
    types.StructField("Zone",types.StringType(),True),
    types.StructField("service_zone",types.StringType(),True)
])

In [65]:
df_zones = spark.read \
    .option("header","true") \
    .schema(zones_schema) \
    .csv('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework/taxi+_zone_lookup.csv')

In [75]:
df_zones = df_zones.repartition(4) #partitioning file into smaller chuncks so that executors in spark cluster can process file efficiently

In [76]:
df_zones.write.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework/zonesparquet/')

In [77]:
df_zones_parquet = spark.read.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/Spark/homework/zonesparquet/')

In [78]:
df_zones_parquet.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [79]:
df_zones_parquet.registerTempTable('zones')

In [80]:
#Q6
df_fhvhv_zones=spark.sql("""
SELECT 
    
    cast(b.Zone ||'/'|| c.Zone as CHAR(100)) as Pickup_Dropoff_Zone,
    count(1) as num_of_trips
    
    
FROM fhvhv_feb a

LEFT JOIN zones b
on a.PULocationID = b.LocationID

LEFT JOIN zones c
on a.DOLocationID = c.LocationID

GROUP BY 1
ORDER BY 2 DESC

""").show()

+--------------------+------------+
| Pickup_Dropoff_Zone|num_of_trips|
+--------------------+------------+
|East New York/Eas...|       45041|
|Borough Park/Boro...|       37329|
|   Canarsie/Canarsie|       28026|
|Crown Heights Nor...|       25976|
| Bay Ridge/Bay Ridge|       17934|
|Jackson Heights/J...|       14688|
|     Astoria/Astoria|       14688|
|Central Harlem No...|       14481|
|Bushwick South/Bu...|       14424|
|Flatbush/Ditmas P...|       13976|
|South Ozone Park/...|       13716|
|Brownsville/Brown...|       12829|
|      JFK Airport/NA|       12542|
|Prospect-Lefferts...|       11814|
|Forest Hills/Fore...|       11548|
|Bushwick North/Bu...|       11491|
|Bushwick South/Bu...|       11487|
|Crown Heights Nor...|       11462|
|Crown Heights Nor...|       11342|
|Prospect-Lefferts...|       11308|
+--------------------+------------+
only showing top 20 rows



In [98]:
#Q6 - second option
df_Q6_option2=spark.sql("""
SELECT 
    
    PULocationID,
    DOLocationID,
    count(1) as num_of_trips
    
    
FROM fhvhv_feb a

GROUP BY 1,2

""")

In [100]:
df_zones_parquet_tmp1 = df_zones_parquet \
                        .withColumnRenamed('LocationID', 'PULocationID')

df_zones_parquet_tmp2 = df_zones_parquet \
                        .withColumnRenamed('LocationID', 'DOLocationID')

In [105]:
df_Q6_option2.join(df_zones_parquet_tmp1, ['PULocationID'], "left") \
             .join(df_zones_parquet_tmp2, ['DOLocationID'], "left") \
             .sort(df_Q6_option2.num_of_trips.desc()) \
             .show()

+------------+------------+------------+---------+--------------------+------------+---------+--------------------+------------+
|DOLocationID|PULocationID|num_of_trips|  Borough|                Zone|service_zone|  Borough|                Zone|service_zone|
+------------+------------+------------+---------+--------------------+------------+---------+--------------------+------------+
|          76|          76|       45041| Brooklyn|       East New York|   Boro Zone| Brooklyn|       East New York|   Boro Zone|
|          26|          26|       37329| Brooklyn|        Borough Park|   Boro Zone| Brooklyn|        Borough Park|   Boro Zone|
|          39|          39|       28026| Brooklyn|            Canarsie|   Boro Zone| Brooklyn|            Canarsie|   Boro Zone|
|          61|          61|       25976| Brooklyn| Crown Heights North|   Boro Zone| Brooklyn| Crown Heights North|   Boro Zone|
|          14|          14|       17934| Brooklyn|           Bay Ridge|   Boro Zone| Brooklyn|   

In [99]:
#***Does not work***
df_Q6_option2.as("a").join(df_zones_parquet.as("b"), $"a.PULocationID" == $"b.LocationID", "left") \
                     .join(df_zones_parquet.as("c"), $"a.DOLocationID" == $"c.LocationID", "left") \
                     .show()

SyntaxError: invalid syntax (<ipython-input-99-30ea790be275>, line 1)

In [49]:
SparkSession.stop(spark)